In [1]:
import os
import re
import glob
import cutlet

from tqdm.notebook import tqdm
import pandas as pd

import tensorflow as tf

import sentencepiece as spm

from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

from transformers import T5Tokenizer, TFT5ForConditionalGeneration

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

main_dir = "Datasets\OPUS100-dataset"

katsu = cutlet.Cutlet()
katsu.use_foreign_spelling = False

JA_unicode = []
with open("E:\Datasets\ASR-dataset\JA_unicode.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        line = line.split()[1:]
        for char in line:
            JA_unicode.append(char)

# Prep

In [2]:
def clean_kanji(sentence):
    symbols = f"[^{JA_unicode}]"
    sentence = re.sub(symbols, "", sentence).strip()
    symbols = f"[（.*?）！-～.,;..._。、-〿・■（）：ㇰ-ㇿ㈠-㉃㊀-㋾㌀-㍿「」『』→ー -~‘–※π—ゐ’“”]"
    sentence = re.sub(symbols, "", sentence).strip()
    return sentence

def clean_romaji(sentence):
    sentence = sentence.strip().lower()
    sentence = re.sub(r'[.,"\'\/?]', "", sentence)
    sentence = sentence.split()
    for i, mora in enumerate(sentence):
        if (mora == "n") | (mora == "u") & (i < len(sentence) - 1):
            prev_mora = sentence.pop(i-1)
            sentence[i-1] = "".join([prev_mora, mora])
    sentence = " ".join(sentence)
    return sentence

In [3]:
# data = pd.read_csv("E:\Datasets\ASR-dataset\ASRDataset.csv")
# data = data['romaji'].to_list()
# data = [x.replace("|", " ") + "\n" for x in data]
# with open(r"E:\Datasets\ASR-dataset\tokenizer_text\asr_text.txt", "w") as f:
#     f.writelines(data)

In [4]:
# paths = glob.glob(f"{main_dir}/*.ja")
# paths

# for path in paths:
#     lines = []
#     with open(path, "r", encoding="utf-8") as f:
#         for line in tqdm(f.readlines()):
#             try:
#                 new_line = clean_kanji(line)
#                 new_line = katsu.romaji(new_line)
#                 new_line = clean_romaji(new_line)
#             except:
#                 continue
#             if new_line != "":
#                 lines.append(new_line + "\n")
#     new_path = os.path.join(r"E:\Datasets\ASR-dataset\tokenizer_text", path.split("\\")[-1])
#     with open(new_path, "w", encoding="utf-8") as f:
#         f.writelines(lines)

In [5]:
path = "D:\School-stuff\Sem-2\PR-Project\HoloASR\Datasets\Tatoeba-dataset\Tatoeba.en-ja.ja"

with open(path, "r", encoding="utf-8") as f:
    lines = f.readlines()

new_lines = []
for line in tqdm(lines, total=len(lines)):
    try:
        new_line = clean_kanji(line)
        new_line = katsu.romaji(new_line)
        new_line = clean_romaji(new_line)
    except:
        continue
    if new_line != "":
        new_lines.append(new_line + "\n")

new_path = os.path.join(r"E:\Datasets\ASR-dataset\tokenizer_text", path.split("\\")[-1])
with open(new_path, "w", encoding="utf-8") as f:
    f.writelines(new_lines)

  0%|          | 0/208866 [00:00<?, ?it/s]

# Training